In [217]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import json
import requests
import json


handgun = "https://www.luckygunner.com/handgun"

In [218]:
# returns 2 list of http links separated by availability
def luckyGunner_availability(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    page_body = soup.body
    products = soup.find_all('div', class_='main')

    
    # list of all handgun ammo seperated by availability
    products = soup.find_all('div', class_='block-content block-no-padding')
    bulletsList = str(products).split('<div class="main-category-cols">')
    inStock = bulletsList[1]
    outofStock = bulletsList[2]

    inStockScrapedLinks = []
    outofStockScrapedLinks = []
    
    inStock = inStock.split("</li>")
    for inStockItems in inStock:
        inStockScrapedLinks.append(re.findall("(?P<url>https?://[^\s]+)", str(inStockItems)))


    outofStock = outofStock.split("</li>")
    for outofStockItems in outofStock:
        outofStockScrapedLinks.append(re.findall("(?P<url>https?://[^\s]+)", str(outofStockItems)))

    inStockCleanLinks = []
    outofStockCleanLinks = []
    

    for i in inStockScrapedLinks:
        cleanLink = ""
        for j in i:
            for k in j:
                if k != '"':
                    cleanLink += k
                    
                else:
                    inStockCleanLinks.append(cleanLink)
                    break
                
    for i in outofStockScrapedLinks:
        cleanLink = ""
        for j in i:
            for k in j:
                if k != '"':
                    cleanLink += k
                else:
                    outofStockCleanLinks.append(cleanLink)
                    break
    return inStockCleanLinks, outofStockCleanLinks

In [219]:
x = luckyGunner_availability(handgun)

In [220]:
def scrapeHandGunAmmo(link):
    link+="?limit=all"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    html_response = requests.get(url=link, headers = headers)
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")

    names = []
    prices = []

    # list of html elements
    names_list = soup.find_all("h3", class_="product-name")
    prices_list = soup.find_all("div", class_="price-box")


    
    for i in prices_list:
        if "Special Price" in str(i):
            sp = i.find("p", class_="special-price")
            sp = sp.find("span", class_="price")
           
            num = ""
            for j in sp.text:
                if str(j) != " " and str(j) != "$":
                    num += str(j)
            prices.append(num)
            num = ""      

        else:
            num = ""
            for j in i.find("span").text:
                if str(j) != " " and str(j) != "$":
                    num += str(j)
            prices.append(num)
            num = ""
                 
                 
    for i in  names_list:
        names.append(i.text)

    try:
        ammoData = pd.DataFrame(
            {"name": names, "price": prices}
        )
        return ammoData
        
    except:
        print("Check link")

In [221]:
list_of_names = []
list_of_prices = []
list_of_links = []

for i in x[0]:
    names = []
    prices = []
    y = scrapeHandGunAmmo(i)
    for j in y["name"]:
        names.append(j.replace("\n\n", ""))
    list_of_names.append(names)
    

    for j in y["price"]:
        prices.append(j.replace("\n", ""))

    list_of_prices.append(prices)



In [225]:
for i in list_of_names:
    print(i)
    print()
print("======================================================================================")
for i in list_of_prices:
    print(i)
    print()

['30 Luger - 93 gr SJSP - Fiocchi - 50 Rounds']

['32 ACP - 71 Grain FMJ - PMC - 50 Rounds', '32 ACP - 73 Grain FMJ - Fiocchi - 50 Rounds']

['38 Special - 158 Grain LRN - Magtech - 50 Rounds', '38 Special - +P 120 Grain JHP - Federal Punch - 20 Rounds', '38 Special - 125 Grain JHP - Speer Gold Dot - 50 Rounds']

['380 Auto - 92 Grain FMJ - Sellier & Bellot - 50 Rounds', '380 Auto - 95 Grain FMJ - Federal American Eagle- 50 Rounds ', '380 Auto - 95 Grain FMJ - Blazer Brass - 50 Rounds ', '380 Auto - 75 Grain Frangible HP - SinterFire Special Duty - 20 Rounds', '380 Auto - 88 gr JHP - Remington UMC- 100 Rounds ', '380 ACP - 90 Grain XTP JHP - Hornady American Gunner - 25 Rounds', '380 Auto - 90 Grain JHP XTP - Underwood - 20 Rounds', '380 Auto - 90 Grain FTX - Hornady Critical Defense - 25 Rounds ']

['40 S&W - 180 Grain FMJ - Blazer Brass - 1000 Rounds', '40 S&W - 165 Grain TMJ - Speer LAWMAN - 1000 Rounds', '40 S&W - 165 Grain FMJ - Blazer Brass - 1000 Rounds', '40 S&W - 125 Grain Fra